<a href="https://colab.research.google.com/github/6Diksha/Language_Detection/blob/main/Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#asking the path of the csv file that contains the dataset of tweets as input from user(the csv file is stored on the drive)
path_tweet = input("Enter path to the Tweets Dataset[csv file]: ")  
data_tweet = pd.read_csv(path_tweet) #reading the csv file 

#asking the heading of the column in the csv file under which all the tweets were listed as input from the user
column_heading = input("Enter the heading of the column that contains the tweets(case-sensitive): ")

saved_column = data_tweet[column_heading] #saving only the specific column which contains the tweets as another variable

In [ ]:
#importing the language detection dataset(for 17 languages) from github(raw data)
#a csv file from github(raw data) can be imported by pasting the url of that page(in Google Colab)
url = "https://raw.githubusercontent.com/basil-b2s/Language-Detector/master/language_detection.csv"
data_lang = pd.read_csv(url) #reading the csv file

In [ ]:
#assigning the sample text and its corresponding language from the language dataset to different variables
dataset_text = data_lang["Text"] #independent variable
dataset_language = data_lang["Language"] #dependent varaiable

In [ ]:
#encoding the labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset_language = le.fit_transform(dataset_language)

In [ ]:
#preprocessing the data/formatting the data
data_formatted = []
 
for text in dataset_text:
  text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text) #removing numbers and special characters
  text = re.sub(r'[[]]', ' ', text)
  text = text.lower() #converting the data to lowercase
  data_formatted.append(text)

In [ ]:
#converting text using features into a vector
#BoW Model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() #assigning short name
dataset_text = cv.fit_transform(data_formatted).toarray() #an array of frequency of features
dataset_text.shape  

In [ ]:
from sklearn.model_selection import train_test_split
dataset_text_train, dataset_text_test, dataset_language_train, dataset_language_test = train_test_split(dataset_text, dataset_language, test_size = 0.20) #the ratio/size is 0.20

In [ ]:
#using Naive-Bayes model
#model creation
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB() 
model.fit(dataset_text_train, dataset_language_train) #fitting into the NB model

In [ ]:
dataset_language_pred = model.predict(dataset_text_test) 

In [ ]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(dataset_language_test, dataset_language_pred) #prediction of accuracy
print("Accuracy: ", ac)

In [ ]:
from sklearn.metrics import confusion_matrix

plt.figure(figsize = (15, 15))
cm = confusion_matrix(dataset_language_test, dataset_language_pred)
sns.heatmap(cm, annot = True)
plt.show()

In [ ]:
def predict(text): #text variable is the text in the tweet
  dataset_text = cv.transform([text]).toarray() #BoW
  lang = model.predict(dataset_text) #Naive-Bayes
  lang = le.inverse_transform(lang) #transformed back to original language
  print("The language is:", lang[0])

In [ ]:
lower = int(input("Enter lower limit: ")) #entering the range
upper = int(input("Enter upper limit: "))
for i in saved_column[0:100]: #iterating over a specific range, in order to avoid any runtime errors
  print(i)
  predict(i)